# Entrenamiento Multilabel con PASCAL VOC 2007

In [13]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss, f1_score, precision_score, recall_score, accuracy_score

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {len(tf.config.list_physical_devices('GPU')) > 0}")

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

TensorFlow version: 2.20.0
GPU disponible: False


In [14]:
PROJECT_ROOT = Path(os.getcwd()).parent
DATA_DIR = PROJECT_ROOT / 'data' / 'voc2007'
MODELS_DIR = PROJECT_ROOT / 'models'
MODELS_DIR.mkdir(parents=True, exist_ok=True)

IMG_SIZE = (224, 224)
BATCH_SIZE = 16
INITIAL_EPOCHS = 30
FINETUNING_EPOCHS = 40
LEARNING_RATE_INITIAL = 0.0005
LEARNING_RATE_FINETUNING = 0.00005

print(f"Configuracion:")
print(f"  Tamaño imagen: {IMG_SIZE}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Epocas inicial: {INITIAL_EPOCHS}")
print(f"  Epocas fine-tuning: {FINETUNING_EPOCHS}")

Configuracion:
  Tamaño imagen: (224, 224)
  Batch size: 16
  Epocas inicial: 30
  Epocas fine-tuning: 40


In [15]:
print(f"Cargando desde: {DATA_DIR}")

with open(DATA_DIR / 'classes.json', 'r') as f:
    classes = json.load(f)

NUM_CLASSES = len(classes)

print(f"Clases cargadas: {NUM_CLASSES}")
print(f"Primeras 10 clases: {classes[:10]}")

Cargando desde: c:\Users\mlata\Documents\iajordy2\data\voc2007
Clases cargadas: 20
Primeras 10 clases: ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow']


In [16]:
print("Cargando dataset PASCAL VOC 2007 desde NPZ...")

# Cargar NPZ
npz_file = DATA_DIR / 'voc2007_multilabel.npz'
if not npz_file.exists():
    raise FileNotFoundError(f"No se encuentra {npz_file}. Ejecuta primero 01_data_analysis.ipynb")

data = np.load(npz_file)
images = data['images']
labels = data['labels']

print(f"Imagenes cargadas: {images.shape}")
print(f"Labels cargados: {labels.shape}")
print(f"Clases por imagen (promedio): {labels.sum(axis=1).mean():.2f}")

# Normalizar imagenes a [0, 1]
images = images.astype(np.float32) / 255.0

print(f"Imagenes normalizadas a rango [0, 1]")

Cargando dataset PASCAL VOC 2007 desde NPZ...
Imagenes cargadas: (2501, 224, 224, 3)
Labels cargados: (2501, 20)
Clases por imagen (promedio): 1.61
Imagenes normalizadas a rango [0, 1]


In [17]:
X_train, X_temp, y_train, y_temp = train_test_split(
    images, labels, test_size=0.3, random_state=SEED
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=SEED
)

print(f"Train: {len(X_train)} imágenes")
print(f"Val: {len(X_val)} imágenes")
print(f"Test: {len(X_test)} imágenes")

print(f"Train labels: {y_train.sum(axis=1).mean():.2f} categorías/imagen")
print(f"Test labels: {y_test.sum(axis=1).mean():.2f} categorías/imagen")
print(f"Val labels: {y_val.sum(axis=1).mean():.2f} categorías/imagen")

Train: 1750 imágenes
Val: 375 imágenes
Test: 376 imágenes
Train labels: 1.62 categorías/imagen
Test labels: 1.59 categorías/imagen
Val labels: 1.57 categorías/imagen


In [ ]:
# Calcular pesos por clase para combatir desbalance
pos_counts = y_train.sum(axis=0)
neg_counts = y_train.shape[0] - pos_counts

# Peso positivo = negativos / positivos (LIMITADO a max 3)
pos_weight = (neg_counts + 1e-6) / (pos_counts + 1e-6)
pos_weight = np.clip(pos_weight, 1.0, 3.0)  # Max 3 para evitar over-compensación

class_weights = tf.constant(pos_weight, dtype=tf.float32)

print("Pesos por clase calculados (limitados a max 3)")
print(f"  Min: {pos_weight.min():.2f}")
print(f"  Max: {pos_weight.max():.2f}")
print(f"  Media: {pos_weight.mean():.2f}")

# Binary Cross-Entropy Weighted (más estable que Focal Loss)
def weighted_bce_loss(y_true, y_pred):
    """
    Binary Cross-Entropy con class weights.
    Más estable que Focal Loss para evitar predecir todo como positivo.
    """
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
    
    # BCE componentes
    bce = -(y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))
    
    # Aplicar class weights solo a positivos
    weighted_bce = bce * (y_true * class_weights + (1 - y_true) * 1.0)
    
    return tf.reduce_mean(weighted_bce)

print("Weighted BCE Loss definida (más estable que Focal Loss)")


Pesos por clase calculados (limitados a max 10)
  Min: 1.30
  Max: 10.00
  Media: 9.20
Focal Loss con class weights definida (gamma=2.0)


In [19]:
# Data augmentation para training
train_datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

# Sin augmentation para val/test (ya están normalizadas)
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Fit datagen en datos de train
train_datagen.fit(X_train)

print("Generadores de datos creados")
print(f"  Train samples: {len(X_train)}")
print(f"  Val samples: {len(X_val)}")
print(f"  Test samples: {len(X_test)}")
print(f"  Batch size: {BATCH_SIZE}")

Generadores de datos creados
  Train samples: 1750
  Val samples: 375
  Test samples: 376
  Batch size: 16


In [20]:
def create_multilabel_model(num_classes, img_size=(224, 224)):
    inputs = layers.Input(shape=(*img_size, 3))
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=inputs)
    base_model.trainable = False
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    return model, base_model

model, base_model = create_multilabel_model(NUM_CLASSES, IMG_SIZE)
print(f"Modelo creado")
print(f"Total parametros: {model.count_params():,}")

Modelo creado
Total parametros: 4,841,911


In [ ]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=LEARNING_RATE_INITIAL),
    loss=weighted_bce_loss,  # Cambiado de focal_loss a weighted_bce_loss
    metrics=[
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc', multi_label=True)
    ]
)

callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, verbose=1),
    keras.callbacks.ModelCheckpoint(filepath=str(MODELS_DIR / 'model_phase1_best.h5'), monitor='val_loss', save_best_only=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-8, verbose=1)
]

print(f"Modelo compilado - FASE 1: Training inicial con Weighted BCE Loss")

history_phase1 = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=INITIAL_EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

print(f"Fase 1 completada")


Modelo compilado - FASE 1: Training inicial con Focal Loss
Epoch 1/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - auc: 0.4734 - loss: 0.2623 - precision: 0.1091 - recall: 0.3950
Epoch 1: val_loss improved from None to 0.24407, saving model to c:\Users\mlata\Documents\iajordy2\models\model_phase1_best.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 29s 215ms/step - auc: 0.4928 - loss: 0.2557 - precision: 0.1145 - recall: 0.3543 - val_auc: 0.5011 - val_loss: 0.2441 - val_precision: 0.2262 - val_recall: 0.4312 - learning_rate: 5.0000e-04
Epoch 2/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - auc: 0.4901 - loss: 0.2488 - precision: 0.1323 - recall: 0.3107
Epoch 2: val_loss improved from 0.24407 to 0.24401, saving model to c:\Users\mlata\Documents\iajordy2\models\model_phase1_best.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 23s 205ms/step - auc: 0.4952 - loss: 0.2479 - precision: 0.1349 - recall: 0.2759 - val_auc: 0.5046 - val_loss: 0.2440 - val_precision: 0.2231 - val_recall: 0.4228 - learning_rate: 5.0000e-04
Epoch 3/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - auc: 0.4866 - loss: 0.2465 - precision: 0.1505 - recall: 0.2287
Epoch 3: val_loss improved from 0.24401 to 0.24299, saving model to c:\Users\mlata\Documents\iajordy2\models\model_phase1_best.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 24s 221ms/step - auc: 0.4908 - loss: 0.2466 - precision: 0.1618 - recall: 0.2303 - val_auc: 0.5009 - val_loss: 0.2430 - val_precision: 0.3956 - val_recall: 0.2767 - learning_rate: 5.0000e-04
Epoch 4/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - auc: 0.4855 - loss: 0.2472 - precision: 0.1910 - recall: 0.2551
Epoch 4: val_loss improved from 0.24299 to 0.24157, saving model to c:\Users\mlata\Documents\iajordy2\models\model_phase1_best.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 25s 231ms/step - auc: 0.5030 - loss: 0.2451 - precision: 0.1892 - recall: 0.2413 - val_auc: 0.5011 - val_loss: 0.2416 - val_precision: 0.2880 - val_recall: 0.3667 - learning_rate: 5.0000e-04
Epoch 5/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - auc: 0.4906 - loss: 0.2441 - precision: 0.2229 - recall: 0.2676
Epoch 5: val_loss did not improve from 0.24157
110/110 ━━━━━━━━━━━━━━━━━━━━ 25s 223ms/step - auc: 0.4975 - loss: 0.2456 - precision: 0.2129 - recall: 0.2360 - val_auc: 0.4974 - val_loss: 0.2434 - val_precision: 0.2880 - val_recall: 0.3667 - learning_rate: 5.0000e-04
Epoch 6/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - auc: 0.4959 - loss: 0.2438 - precision: 0.1967 - recall: 0.1587
Epoch 6: val_loss did not improve from 0.24157
110/110 ━━━━━━━━━━━━━━━━━━━━ 22s 199ms/step - auc: 0.4910 - loss: 0.2452 - precision: 0.1895 - recall: 0.1505 - val_auc: 0.5001 - val_loss: 0.2416 - val_precision: 0.1253 - val_recall: 0.1596 - learning_rate: 5.0000e-04
Epoch 

110/110 ━━━━━━━━━━━━━━━━━━━━ 22s 199ms/step - auc: 0.4956 - loss: 0.2443 - precision: 0.2255 - recall: 0.1932 - val_auc: 0.5000 - val_loss: 0.2414 - val_precision: 0.0960 - val_recall: 0.0611 - learning_rate: 5.0000e-04
Epoch 8/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - auc: 0.4921 - loss: 0.2421 - precision: 0.2314 - recall: 0.1678
Epoch 8: val_loss did not improve from 0.24143
110/110 ━━━━━━━━━━━━━━━━━━━━ 22s 198ms/step - auc: 0.4988 - loss: 0.2437 - precision: 0.2303 - recall: 0.1568 - val_auc: 0.5000 - val_loss: 0.2416 - val_precision: 0.0960 - val_recall: 0.0611 - learning_rate: 5.0000e-04
Epoch 9/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - auc: 0.4900 - loss: 0.2450 - precision: 0.2328 - recall: 0.2053
Epoch 9: val_loss improved from 0.24143 to 0.24066, saving model to c:\Users\mlata\Documents\iajordy2\models\model_phase1_best.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 22s 200ms/step - auc: 0.4941 - loss: 0.2435 - precision: 0.2365 - recall: 0.2268 - val_auc: 0.5000 - val_loss: 0.2407 - val_precision: 0.2587 - val_recall: 0.3294 - learning_rate: 5.0000e-04
Epoch 10/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - auc: 0.4890 - loss: 0.2428 - precision: 0.2302 - recall: 0.2233
Epoch 10: val_loss improved from 0.24066 to 0.24048, saving model to c:\Users\mlata\Documents\iajordy2\models\model_phase1_best.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 22s 203ms/step - auc: 0.4864 - loss: 0.2435 - precision: 0.2363 - recall: 0.2229 - val_auc: 0.5000 - val_loss: 0.2405 - val_precision: 0.2240 - val_recall: 0.4278 - learning_rate: 5.0000e-04
Epoch 11/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - auc: 0.4920 - loss: 0.2433 - precision: 0.2178 - recall: 0.2141
Epoch 11: val_loss did not improve from 0.24048
110/110 ━━━━━━━━━━━━━━━━━━━━ 23s 206ms/step - auc: 0.4962 - loss: 0.2432 - precision: 0.2193 - recall: 0.2123 - val_auc: 0.5000 - val_loss: 0.2406 - val_precision: 0.2880 - val_recall: 0.3667 - learning_rate: 5.0000e-04
Epoch 12/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - auc: 0.4884 - loss: 0.2441 - precision: 0.2448 - recall: 0.2474
Epoch 12: val_loss improved from 0.24048 to 0.24035, saving model to c:\Users\mlata\Documents\iajordy2\models\model_phase1_best.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 22s 201ms/step - auc: 0.4882 - loss: 0.2432 - precision: 0.2496 - recall: 0.2384 - val_auc: 0.5000 - val_loss: 0.2404 - val_precision: 0.4213 - val_recall: 0.2683 - learning_rate: 5.0000e-04
Epoch 13/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - auc: 0.4881 - loss: 0.2398 - precision: 0.2693 - recall: 0.2411
Epoch 13: val_loss did not improve from 0.24035
110/110 ━━━━━━━━━━━━━━━━━━━━ 22s 199ms/step - auc: 0.4937 - loss: 0.2430 - precision: 0.2650 - recall: 0.2310 - val_auc: 0.5000 - val_loss: 0.2405 - val_precision: 0.2587 - val_recall: 0.3294 - learning_rate: 5.0000e-04
Epoch 14/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - auc: 0.4903 - loss: 0.2403 - precision: 0.2355 - recall: 0.2499
Epoch 14: val_loss did not improve from 0.24035
110/110 ━━━━━━━━━━━━━━━━━━━━ 22s 200ms/step - auc: 0.4867 - loss: 0.2431 - precision: 0.2217 - recall: 0.2296 - val_auc: 0.5000 - val_loss: 0.2405 - val_precision: 0.2587 - val_recall: 0.3294 - learning_rate: 5.0000e-04
Ep

In [ ]:
base_model.trainable = True
fine_tune_at = len(base_model.layers) - 40
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=optimizers.Adam(learning_rate=LEARNING_RATE_FINETUNING),
    loss=weighted_bce_loss,  # Cambiado de focal_loss a weighted_bce_loss
    metrics=[
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc', multi_label=True)
    ]
)

train_datagen_ft = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    vertical_flip=False
)

print(f"FASE 2: Fine-tuning con ultimas {len(base_model.layers) - fine_tune_at} capas descongeladas")

history_phase2 = model.fit(
    train_datagen_ft.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=FINETUNING_EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

print(f"Fase 2 completada")


FASE 2: Fine-tuning con ultimas 40 capas descongeladas
Epoch 1/40
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - auc: 0.4768 - loss: 0.2639 - precision: 0.1856 - recall: 0.2744
Epoch 1: val_loss did not improve from 0.24035
110/110 ━━━━━━━━━━━━━━━━━━━━ 34s 238ms/step - auc: 0.4829 - loss: 0.2529 - precision: 0.1991 - recall: 0.2833 - val_auc: 0.5074 - val_loss: 0.2404 - val_precision: 0.4213 - val_recall: 0.2683 - learning_rate: 5.0000e-05
Epoch 2/40
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - auc: 0.4861 - loss: 0.2437 - precision: 0.2146 - recall: 0.2719
Epoch 2: val_loss did not improve from 0.24035
110/110 ━━━━━━━━━━━━━━━━━━━━ 25s 224ms/step - auc: 0.4942 - loss: 0.2447 - precision: 0.2186 - recall: 0.2674 - val_auc: 0.4966 - val_loss: 0.2404 - val_precision: 0.4213 - val_recall: 0.2683 - learning_rate: 5.0000e-05
Epoch 3/40
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - auc: 0.5066 - loss: 0.2446 - precision: 0.2187 - recall: 0.2550
Epoch 3: val_loss did not improve from 0.24035
110/

In [ ]:
y_val_pred = model.predict(X_val, verbose=1)

# Buscar umbral optimo global por F1-micro (rango AMPLIADO 0.3-0.7)
thresholds = np.arange(0.3, 0.75, 0.05)
f1_scores = []
for thresh in thresholds:
    y_val_pred_binary = (y_val_pred >= thresh).astype(int)
    f1 = f1_score(y_val, y_val_pred_binary, average='micro', zero_division=0)
    f1_scores.append(f1)

best_idx = int(np.argmax(f1_scores))
best_threshold = float(thresholds[best_idx])
print(f"Threshold optimo (global): {best_threshold:.2f}")
print(f"F1-micro max (global): {f1_scores[best_idx]:.4f}")

# Umbral optimo por clase (rango ajustado)
best_thresholds = []
for c in range(NUM_CLASSES):
    f1_c = []
    for thresh in thresholds:
        pred_c = (y_val_pred[:, c] >= thresh).astype(int)
        f1_c.append(f1_score(y_val[:, c], pred_c, average='binary', zero_division=0))
    best_thresholds.append(float(thresholds[int(np.argmax(f1_c))]))

best_thresholds = np.array(best_thresholds)
print(f"Thresholds por clase (promedio): {best_thresholds.mean():.2f}")
print(f"Thresholds por clase (min-max): {best_thresholds.min():.2f} - {best_thresholds.max():.2f}")

# Metricas finales con thresholds por clase
y_val_pred_binary = (y_val_pred >= best_thresholds).astype(int)
positive_rate = y_val_pred_binary.mean()
metrics_phase2 = {
    'hamming_loss': hamming_loss(y_val, y_val_pred_binary),
    'subset_accuracy': accuracy_score(y_val, y_val_pred_binary),
    'f1_micro': f1_score(y_val, y_val_pred_binary, average='micro', zero_division=0),
    'f1_macro': f1_score(y_val, y_val_pred_binary, average='macro', zero_division=0),
    'f1_samples': f1_score(y_val, y_val_pred_binary, average='samples', zero_division=0),
    'precision_micro': precision_score(y_val, y_val_pred_binary, average='micro', zero_division=0),
    'recall_micro': recall_score(y_val, y_val_pred_binary, average='micro', zero_division=0),
}

print("\nMETRICAS FINALES EN VALIDACION")
for metric, value in metrics_phase2.items():
    print(f"{metric}: {value:.4f}")
print(f"Tasa de positivos predichos: {positive_rate:.4f}")
print(f"Tasa de positivos reales: {y_val.mean():.4f}")


12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 353ms/step
Threshold optimo (global): 0.50
F1-micro max (global): 0.3278
Thresholds por clase (promedio): 0.20
Thresholds por clase (min-max): 0.20 - 0.20

METRICAS FINALES EN VALIDACION
hamming_loss: 0.9215
subset_accuracy: 0.0000
f1_micro: 0.1456
f1_macro: 0.1365
precision_micro: 0.0785
recall_micro: 1.0000
Tasa de positivos predichos: 1.0000


In [24]:
model.save(MODELS_DIR / 'voc_multilabel_final.h5')
model.save(MODELS_DIR / 'voc_multilabel_final.keras')
print(f"Modelo guardado")

with open(MODELS_DIR / 'training_results.json', 'w') as f:
    json.dump({
        'metrics': metrics_phase2,
        'config': {
            'initial_epochs': INITIAL_EPOCHS,
            'finetuning_epochs': FINETUNING_EPOCHS,
            'batch_size': BATCH_SIZE,
            'img_size': IMG_SIZE,
            'learning_rate_initial': LEARNING_RATE_INITIAL,
            'learning_rate_finetuning': LEARNING_RATE_FINETUNING
        },
        'thresholds': best_thresholds.tolist()
    }, f, indent=2)
print(f"Resultados guardados")

Modelo guardado
Resultados guardados
